In [1]:
!pip install pystan

In [2]:
stan_model = """
data {
  int<lower=0> N;       // Number of data points
  vector[N] x;          // Predictor variable
  vector[N] y;          // Output variable
}

parameters {
  real alpha1;                    // Intercept for the first normal distribution
  real alpha2;                    // Intercept for the second normal distribution
  real beta1;                     // Slope for the first normal distribution
  real beta2;                     // Slope for the second normal distribution
  real<lower=0> sigma1;           // Standard deviation for the first normal distribution
  real<lower=0> sigma2;           // Standard deviation for the second normal distribution
  vector<lower=0, upper=1>[N] z_prob; // The probability of z being 1 for each data point
}

model {
  // Priors
  beta1 ~ normal(0, 1);
  beta2 ~ normal(0, 1);
  alpha1 ~ normal(0, 1);
  alpha2 ~ normal(0, 1);
  sigma1 ~ inv_gamma(1, 1);
  sigma2 ~ inv_gamma(1, 1);

  // Likelihood
  for (n in 1:N) {
    // Model z as a bernoulli distributed variable based on its probability
    target += bernoulli_lpmf(1 | z_prob[n]) * normal_lpdf(y[n] | beta1 * x[n] + alpha1, sigma1)
              + bernoulli_lpmf(0 | z_prob[n]) * normal_lpdf(y[n] | beta2 * x[n] + alpha2, sigma2);
  }
}


"""


In [3]:
import pandas as pd
import cmdstanpy
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv('./regression.csv')

# Display the first few rows of the dataframe to understand its structure
print(df.head())

x = df['X'].values
y = df['Y'].values
N = len(df)

In [4]:
from cmdstanpy import CmdStanModel

In [5]:
# Compile the model
stan_file = 'mixture_model_2.stan'
with open(stan_file, 'w') as file:
    file.write(stan_model)
model =  cmdstanpy.CmdStanModel(stan_file='./mixture_model_2.stan')

# Prepare data for Stan
stan_data = {'N': N, 'x': x, 'y': y}

# Run the model
fit = model.sample(data=stan_data, iter_sampling=1000, iter_warmup=500, chains=4)

# Extract the results
results = fit.summary()
print(results)

07:48:14 - cmdstanpy - INFO - compiling stan file /Users/siyuwu/558_HW2/mixture_model_2.stan to exe file /Users/siyuwu/558_HW2/mixture_model_2
07:48:26 - cmdstanpy - INFO - compiled model executable: /Users/siyuwu/558_HW2/mixture_model_2
07:48:27 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

07:48:27 - cmdstanpy - INFO - CmdStan done processing.
07:48:27 - cmdstanpy - WARNING - Some chains may have failed to converge.
	Chain 1 had 999 divergent transitions (99.9%)
	Chain 2 had 37 divergent transitions (3.7%)
	Chain 3 had 51 divergent transitions (5.1%)
	Chain 4 had 39 divergent transitions (3.9%)
	Use the "diagnose()" method on the CmdStanMCMC object to see further information.



                    Mean          MCSE        StdDev             5%  \
lp__        2.922390e+05  4.690000e+04  1.224320e+05  129930.000000   
alpha1     -8.059790e-01  1.114730e+00  1.578840e+00      -2.389740   
alpha2     -3.592290e-01  7.732480e-01  1.095180e+00      -1.760160   
beta1       1.025990e+00  8.709080e-01  1.233500e+00      -0.806976   
beta2      -1.267060e+00  5.914490e-01  8.376930e-01      -1.909580   
sigma1     4.470870e+307           inf           inf       0.003468   
sigma2      9.550370e-01  9.071510e-01  1.284830e+00       0.006003   
z_prob[1]   3.474590e-01  1.471470e-01  2.084100e-01       0.000000   
z_prob[2]   3.249330e-01  1.929830e-01  2.733300e-01       0.000000   

                     50%            95%       N_Eff     N_Eff/s        R_hat  
lp__       259125.000000   4.958760e+05     6.81469     43.6839      2.51094  
alpha1         -0.766207   1.736280e+00     2.00602     12.8591  23742.60000  
alpha2          0.392627   9.797030e-01     2.00602

The expectations for each of the normal distributions are:
For the first group: (E1[Y | X = 0.8] = beta1 *0.8 + alpha 1
For the second group: (E2[Y | X = 0.8] = beta 2 * 0.8 + alpha 2)

Given the mean posterior values:
alpha_1 = 0.24985
beta_1 = 0.290827
alpha_2 = 0.255566
beta_2 = 0.284956
z[1] = 0.500603
Z[2] = 0.493170

We can calculate the weighted sum of the expectations:

[E[Y | X = 0.8] =( z[1] + z[2] /2) * (beta_1 *0.8 + alpha_1) + (1 - z[1] +z[2] / 2) * (beta_2 *0.8 + alpha_2)
